In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip '/content/drive/MyDrive/Dataset/Dataset.zip'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Load dataset
train_data_dir = '/content/Dataset/Train'
validation_data_dir = '/content/Dataset/Valid'
test_data_dir = '/content/Dataset/Test'
img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


Found 10002 images belonging to 2 classes.
Found 10002 images belonging to 2 classes.


In [ ]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Evaluate the model
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)

94765736/94765736 [==============================] - 3s 0us/step


Epoch 1/10
312/312 [==============================] - 194s 594ms/step - loss: 1.3737 - accuracy: 0.5002 - val_loss: 0.6931 - val_accuracy: 0.5012
Epoch 2/10
312/312 [==============================] - 176s 566ms/step - loss: 0.6961 - accuracy: 0.5041 - val_loss: 0.6932 - val_accuracy: 0.5005
Epoch 3/10
312/312 [==============================] - 182s 584ms/step - loss: 0.6990 - accuracy: 0.4968 - val_loss: 0.6931 - val_accuracy: 0.5004
Epoch 4/10
312/312 [==============================] - 176s 563ms/step - loss: 0.6968 - accuracy: 0.5020 - val_loss: 0.6923 - val_accuracy: 0.5532
Epoch 5/10
312/312 [==============================] - 174s 559ms/step - loss: 0.6957 - accuracy: 0.4981 - val_loss: 0.6931 - val_accuracy: 0.4997
Epoch 6/10
312/312 [==============================] - 182s 582ms/step - loss: 0.6947 - accuracy: 0.4994 - val_loss: 0.6931 - val_accuracy: 0.4997
Epoch 7/10
312/312 [==============================] - 183s 585ms/step - loss: 0.7032 - accuracy: 0.4955 - val_loss: 0.6931 -

In [ ]:
from sklearn.metrics import classification_report
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

# Get true labels
true_labels = test_generator.classes

# Generate classification report
class_names = list(test_generator.class_indices.keys())
print("Classification Report on Test Data:")
print(classification_report(true_labels, y_pred, target_names=class_names,zero_division=0))

313/313 [==============================] - 39s 121ms/step
Classification Report on Test Data:
              precision    recall  f1-score   support

        Fake       0.50      1.00      0.67      5001
        Real       0.64      0.00      0.00      5001

    accuracy                           0.50     10002
   macro avg       0.57      0.50      0.33     10002
weighted avg       0.57      0.50      0.33     10002



In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(true_labels, y_pred)
print(conf_matrix)


[[4997    4]
 [4994    7]]


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model_c = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
])

# Compile the model
model_c.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model_c.fit(train_generator,
                              steps_per_epoch=train_generator.samples // batch_size,
                              epochs=10,
                              validation_data=validation_generator,
                              validation_steps=validation_generator.samples // batch_size)

# Evaluate the model
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

loss, accuracy = model_c.evaluate(test_generator)
print("Test Accuracy:", accuracy)

Epoch 1/10
312/312 [==============================] - 161s 500ms/step - loss: 0.7134 - accuracy: 0.5410 - val_loss: 0.6128 - val_accuracy: 0.6762
Epoch 2/10
312/312 [==============================] - 150s 480ms/step - loss: 0.6567 - accuracy: 0.6130 - val_loss: 0.6169 - val_accuracy: 0.6325
Epoch 3/10
312/312 [==============================] - 156s 499ms/step - loss: 0.6343 - accuracy: 0.6470 - val_loss: 0.5974 - val_accuracy: 0.7028
Epoch 4/10
312/312 [==============================] - 149s 479ms/step - loss: 0.6131 - accuracy: 0.6678 - val_loss: 0.5006 - val_accuracy: 0.7603
Epoch 5/10
312/312 [==============================] - 156s 500ms/step - loss: 0.5943 - accuracy: 0.6845 - val_loss: 0.5346 - val_accuracy: 0.7463
Epoch 6/10
312/312 [==============================] - 151s 486ms/step - loss: 0.5751 - accuracy: 0.7027 - val_loss: 0.5151 - val_accuracy: 0.7636
Epoch 7/10
312/312 [==============================] - 156s 499ms/step - loss: 0.5662 - accuracy: 0.7055 - val_loss: 0.4794 -

In [ ]:
y_pred_prob = model_c.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

# Get true labels
true_labels = test_generator.classes

# Generate classification report
class_names = list(test_generator.class_indices.keys())
print("Classification Report on Test Data:")
print(classification_report(true_labels, y_pred, target_names=class_names,zero_division=0))

313/313 [==============================] - 20s 62ms/step
Classification Report on Test Data:
              precision    recall  f1-score   support

        Fake       0.70      0.57      0.63      5001
        Real       0.64      0.75      0.69      5001

    accuracy                           0.66     10002
   macro avg       0.67      0.66      0.66     10002
weighted avg       0.67      0.66      0.66     10002



In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(true_labels, y_pred)
print(conf_matrix)

[[2859 2142]
 [1240 3761]]


In [ ]:
model_c.save('cust.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
